In [1]:
import gzip
import json
import pandas as pd
import ast

## TRANSFORMACIONES DE JSON A CSV y DESANIDADO DE COLUMNAS

### Transformación: steam_games

In [8]:
# Ruta del archivo JSON
ruta_archivo = r'M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Originales Comprimidos\steam_games.json.gz'

lista_dicc = []  # Crear una lista vacia para almacenar los objetos JSON

# Descomprimir el archivo con gzip, abrirlo y leer línea por línea
with gzip.open(ruta_archivo, 'rt', encoding='utf-8') as archivo: 
    for line in archivo:
        # Cargar cada línea y agregar a la lista
        data = json.loads(line)
        lista_dicc.append(data)

# Guardar la lista de diccionarios en un DataFrame de PANDAS
df_steam_games = pd.DataFrame(lista_dicc)

# Mostrar el DataFrame
df_steam_games.head()


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_steam_games.to_csv( "steam_games.csv", sep=';', index=False, encoding='utf-8')


### Transformación: australian_user_reviews

In [ ]:

# Ruta del archivo JSON
ruta_archivo = r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Originales Comprimidos\user_reviews.json.gz"


lista_dicc = []  # Crear una lista vacia para almacenar los objetos JSON

with gzip.open(ruta_archivo) as archivo:
    for line in archivo:

            # Se utiliza ast.literal_eval para convertir cada línea en un diccionario
            data = ast.literal_eval(line.decode('utf-8'))
            lista_dicc.append(data)

df_user_reviews = pd.DataFrame(lista_dicc)

df_user_reviews.head()

In [11]:
print((len(df_user_reviews)))

25799


Desanidado de columna Reviews

In [ ]:

# Asegurarse de que la columna 'reviews' sea una lista
df_user_reviews['reviews'] = df_user_reviews['reviews'].apply(lambda x: [ast.literal_eval(str(r)) for r in x])

# Explode la columna 'reviews' para obtener una fila por cada reseña
df_reviews_exploded = df_user_reviews.explode('reviews')

# Crear un nuevo DataFrame a partir de la columna 'reviews'
df_reviews_separated = pd.json_normalize(df_reviews_exploded['reviews'])

# Agregar columnas de user_id y user_url
df_reviews_separated['user_id'] = df_reviews_exploded['user_id'].values
df_reviews_separated['user_url'] = df_reviews_exploded['user_url'].values

#reordenamiento de columnas
column_order = ['user_id', 'user_url'] + [col for col in df_reviews_separated.columns if col not in ['user_id', 'user_url']]
df_reviews_separated = df_reviews_separated[column_order]

# Mostrar el nuevo DataFrame

df_reviews_separated.head()

In [37]:
#Guardar el archivo ya desanidado en csv y parquet para su futuro uso
df_reviews_separated.to_csv( "user_reviews.csv", sep=';', index=False, encoding='utf-8')
df_reviews_separated.to_parquet('user_reviews.parquet', index=False)

### Transformación: australian_user_items

In [15]:

# Ruta del archivo JSON
ruta_archivo = r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Originales Comprimidos\users_items.json.gz"


lista_dicc = []  # Crear una lista vacia para almacenar los objetos JSON

with gzip.open(ruta_archivo) as archivo:
    for line in archivo:
            # Se utiliza ast.literal_eval para convertir cada línea en un diccionario
            data = ast.literal_eval(line.decode('utf-8'))
            lista_dicc.append(data)

df_users_items = pd.DataFrame(lista_dicc)

df_users_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Desanidado de columna Items

In [16]:

# Asegurarse de que la columna 'items' sea una lista
df_users_items['items'] = df_users_items['items'].apply(lambda x: [ast.literal_eval(str(r)) for r in x])

# Explode la columna 'items' para obtener una fila por cada reseña
df_items_exploded = df_users_items.explode('items')

# Crear un nuevo DataFrame a partir de la columna 'items'
df_items_separated = pd.json_normalize(df_items_exploded['items'])  # Corregido aquí

# Agregar columnas de user_id y user_url
df_items_separated['user_id'] = df_items_exploded['user_id'].values
df_items_separated['items_count'] = df_items_exploded['items_count'].values
df_items_separated['steam_id'] = df_items_exploded['steam_id'].values
df_items_separated['user_url'] = df_items_exploded['user_url'].values

# Reordenar columnas
column_order = ['user_id', 'user_url', 'items_count', 'steam_id'] + [col for col in df_items_separated.columns if col not in ['user_id', 'user_url', 'items_count', 'steam_id']]
df_items_separated = df_items_separated[column_order]

# Mostrar el nuevo DataFrame
df_items_separated.head()


,user_id,user_url,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0


In [18]:
#Guardar el archivo ya desanidado en csv para su futuro uso
df_items_separated.to_csv( "user_items.csv", sep=';', index=False, encoding='utf-8')
# df_items_separated.to_parquet('user_items.parquet', index=False)

## TRANSFORMACIONES VARIAS: Steam_Games


In [24]:
import pandas as pd
import numpy as np
import ast

In [3]:
# Especifica la ruta del archivo CSV
ruta_archivo = r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Originales CSV\steam_games.csv"

# Leer el archivo CSV
df_steam = pd.read_csv(ruta_archivo, sep=';', on_bad_lines='skip')

# Mostrar las primeras filas del DataFrame
df_steam.head(5)



C:\Users\Flor Pedraza\AppData\Local\Temp\ipykernel_5196\1818374452.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_steam = pd.read_csv(ruta_archivo, sep=';', on_bad_lines='skip')


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Detallamos info del dataframe
df_steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  object 
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(1), object(12)
memory usage: 11.9+ MB


Elimino columnas no necesarias para el analisis 

In [6]:
# Eliminar las columnas que no se utilizarán 
df_steam = df_steam.drop(columns=['publisher', 'app_name', 'url', 'tags', 'reviews_url', 'specs', 'early_access'])

Manejo de Vacios/Nulos/Duplicados

In [21]:
# Eliminar filas que estén completamente vacías
df_steam = df_steam.dropna(how='all')

# Elimina las filas duplicadas
df_steam = df_steam.drop_duplicates()

df_steam.shape

(32134, 6)

Columna "price":

In [22]:

# Rellenar la columna 'price' con el valor 0 para las filas que contienen 'Free'
df_steam.loc[df_steam['price'].str.contains('Free', case=False, na=False), 'price'] = 0

In [25]:
# Reemplazar valores que no son numéricos por NaN
df_steam['price'] = df_steam['price'].replace(
    [ 'Install Now', 'Play Now', 'Starting at $499.00', 'Starting at $449.00',
      'Play WARMACHINE: Tactics Demo', 'Third-party', 'Play the Demo',
      'Install Theme', '0.5'], np.nan)

In [26]:
# Convertir la columna 'price' a tipo float
df_steam['price'] = pd.to_numeric(df_steam['price'], errors='coerce')

Columna "release_date" y creación de columna "year": 

In [27]:
# Convertir la columna 'release_date' a datetime
df_steam['release_date'] = pd.to_datetime(df_steam['release_date'], errors='coerce')

#Crear columna YEAR apartir de RELASE_DATE
df_steam["year"] = df_steam["release_date"].dt.year

#Convertir a entrero para una mejor visualización
df_steam['year'] = df_steam['year'].fillna(0).astype(int)

df_steam['year'].unique()

array([2018, 2017,    0, 1997, 1998, 2016, 2006, 2005, 2003, 2007, 2002,
       2000, 1995, 1996, 1994, 2001, 1993, 2004, 1999, 2008, 2009, 1992,
       1989, 2010, 2011, 2013, 2012, 2014, 1983, 1984, 2015, 1990, 1988,
       1991, 1985, 1982, 1987, 1981, 1986, 2021, 2019, 1975, 1970, 1980])

Columna "ID": 

In [7]:
#Renombrar columna para futuros usos
df_steam.rename(columns={'id': 'item_id'}, inplace=True)

In [29]:
# Convertir los valores de 'item_id' a enteros, eliminando los decimales
df_steam['item_id'] = df_steam['item_id'].fillna(0).astype(int)

# Convertir la columna 'item_id' a tipo string
df_steam['item_id'] = df_steam['item_id'].astype(str)

# Verifica el cambio
print(df_steam['item_id'].unique())

['761140' '643980' '670290' ... '610660' '658870' '681550']


Columna "genres": 

In [30]:
# Convertir a lista la columna Genres
df_steam['genres'] = df_steam['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [31]:
# count_nan = df_steam['genres'].isna().sum()
# print(f'Cantidad de NaN en la columna "genres": {count_nan}') ->Cantidad de NaN en la columna "genres": 3283

#Reemplazar los NaN por una lista vacía para futuras iteraciones
df_steam['genres'] = df_steam['genres'].apply(lambda x: x if isinstance(x, list) else [])


In [32]:
#ejemplo de df post ETL
df_steam.head()

,genres,title,release_date,price,item_id,developer,year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980,Secret Level SRL,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290,Poolians.com,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,2017
88314,[],NaN,NaT,2.99,773570,NaN,0


In [35]:
#Exportar archivo Steam POST ETL a formato CSV
df_steam.to_csv("steam_post_etl.csv", sep=';', index=False, encoding='utf-8')

## TRANSFORMACIONES VARIAS:  User_reviews

In [36]:
import pandas as pd
import numpy as np

In [38]:
# Especifica la ruta del archivo CSV
ruta_archivo = r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Originales CSV\user_reviews_sentiment_analysis_TextBlob.csv"


# Leer el archivo CSV
df_reviews = pd.read_csv(ruta_archivo, sep=';', on_bad_lines='skip')

# Mostrar las primeras filas del DataFrame
df_reviews.head(5) 

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,It's unique and worth a playthrough.,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
4,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,0


Elimino columnas no necesarias para el analisis 

In [39]:
# Eliminar las columnas que no se utilizarán 
df_reviews = df_reviews.drop(columns=['user_url', 'funny', 'last_edited', 'helpful', 'review',"posted"])
df_reviews.head()

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250.0,True,2
1,76561197970982479,22200.0,True,2
2,76561197970982479,43110.0,True,2
3,js41637,251610.0,True,2
4,js41637,227300.0,True,0


Columna "item_id":

In [40]:
# Convertir los valores de 'item_id' a enteros, eliminando los decimales
df_reviews['item_id'] = df_reviews['item_id'].fillna(0).astype(int)

# Convertir la columna 'item_id' a tipo string
df_reviews['item_id'] = df_reviews['item_id'].astype(str)

In [51]:
#Exportar archivo Reviwes POST ETL a formato CSV
df_reviews.to_csv("reviews_post_etl.csv", sep=';', index=False, encoding='utf-8')

## TRANSFORMACIONES VARIAS:User_items

In [41]:
import pandas as pd
import numpy as np

In [42]:
# Especifica la ruta del archivo CSV
ruta_archivo = r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Originales CSV\user_items.csv"


# Leer el archivo CSV
df_items = pd.read_csv(ruta_archivo, sep=';', on_bad_lines='skip')

# Mostrar las primeras filas del DataFrame
df_items.head(5) 

,user_id,user_url,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,10.0,Counter-Strike,6.0,0.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,30.0,Day of Defeat,7.0,0.0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,40.0,Deathmatch Classic,0.0,0.0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,50.0,Half-Life: Opposing Force,0.0,0.0


Elimino columnas no necesarias para el analisis 

In [43]:
# Eliminar las columnas que no se utilizarán 
df_items = df_items.drop(columns=['user_url', 'items_count', 'steam_id', 'item_name', 'playtime_2weeks'])


Columna "item_id": 

In [44]:
# Convertir los valores de 'item_id' a enteros, eliminando los decimales
df_items['item_id'] = df_items['item_id'].fillna(0).astype(int)

# Convertir la columna 'item_id' a tipo string
df_items['item_id'] = df_items['item_id'].astype(str)



Columna "playtime_forever": 

In [45]:
#convertir a Float
df_items['playtime_forever'] = df_items['playtime_forever'].fillna(0).astype(float)

Agregar columna "price"

In [46]:
# Agregar la columna 'price' al df_items a partir de df_steam utilizando 'item_id' como clave
df_items = df_items.merge(df_steam[['item_id', 'price']], on='item_id', how='left')

In [52]:
#Exportar archivo Items POST ETL a formato CSV
df_items.to_csv("items_post_etl.csv", sep=';', index=False, encoding='utf-8')
df_items.to_parquet("items_post_etl.parquet")

In [2]:
# # Especifica la ruta del archivo CSV
# ruta_archivo = r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\items_post_etl.csv"
# # Leer el archivo CSV
# df_items = pd.read_csv(ruta_archivo, sep=';', on_bad_lines='skip')
# df_items.to_parquet("items_post_etl.parquet")